# 1. Connect Google Earth Engine with Google Cloud Platform (GCP)

This section explains how to create and configure a Google Cloud project for use with **Google Earth Engine (EE)** and **Google Drive** exports.

## 1.1 Create a Google Cloud account
- Go to [https://cloud.google.com](https://cloud.google.com) and sign in with your Google account.  
- If this is your first time, set up billing (you’ll receive free credits by default).  

## 1.2 Create a new project
- Open the [Google Cloud Console](https://console.cloud.google.com/).  
- Click the **project selector** at the top (or press **Ctrl + O**).  
- Choose **New Project** and give it a descriptive name, e.g. **`protofire`**.  
- Wait a few seconds until the new project becomes active.  



## 1.3 Enable required APIs
Within the selected project:

- Press **/** on your keyboard to open the global search bar.  
- Enable the following APIs:
   - **Earth Engine API**  
   - **Google Drive API** *(required for Drive exports)*  

> *If prompted, register for Earth Engine non-commercial use, most research and education users qualify for free access.*

## 1.4 Authenticate Earth Engine in Python

The following code cell will open your default web browser and redirect you to the **Google Earth Engine Notebook Authenticator**.

- Select your **Google account** and your **Cloud Project** (for example, `protofire`).  
- Follow the on-screen instructions until you receive an **Authorization Code**.  
- Return to your notebook, a prompt will appear asking for a **verification code**.  
- Paste the authorization code into that prompt and press **Enter**.

In [2]:
import ee

# Authenticate with the necessary scopes (run once per environment):
ee.Authenticate(scopes=[
    'https://www.googleapis.com/auth/earthengine',          # Earth Engine
    'https://www.googleapis.com/auth/drive',                # Google Drive
    'https://www.googleapis.com/auth/devstorage.read_write' # optional Google Cloud Storage
])

# Initialize EE with your new GCP project ID
ee.Initialize(project='protofire01')

# 2. Tile Grid & Earth Engine Conversion

This section:
1) Builds a latitude/longitude grid over a country boundary (GeoJSON).  
2) Converts the grid to an Earth Engine `FeatureCollection`, enriching tiles with centroid coordinates and SRTM elevation.


**Dependencies:** `geopandas`, `shapely`, `numpy`, `pandas`, `earthengine-api` (and optionally `folium`/`geemap` for quick maps).

In [3]:
# from typing import Optional

import numpy as np
# import pandas as pd
import geopandas as gpd
from shapely.geometry import box
from shapely.ops import unary_union

# Set default CRS for vector work
GEODETIC_CRS = "EPSG:4326"

## 2.1 Grid Generation over a Country Boundary

In [4]:
def make_grid_gdf(country_geojson: str) -> gpd.GeoDataFrame:
    """
    Create a regular lon/lat GEODETIC_CRS (by default EPSG:4326) grid clipped to a country's boundary.

    Parameters
    ----------
    country_geojson : str
        Path to a country boundary GeoJSON/GeoPackage/etc. readable by GeoPandas.
    Returns
    -------
    geopandas.GeoDataFrame
        Grid cells intersecting the country, with a `tile_id` column.
    """
    # The function creates a grid of tiles given a .geojson file that defines the boundary of a country
    country_gdf = (
        gpd.read_file(country_geojson) # Take the geometry of a country as a .geojson file
        .to_crs(GEODETIC_CRS) # Transform geometries to a new Coordinate Reference System (CRS)
        .geometry.union_all()# Returns a geometry containing the union of all geometries
    )

    # Build tile grid
    xmin, ymin, xmax, ymax = country_gdf.bounds
    grid_size_deg = 0.09 # 0.09 degree ~ 10 km grid

    tiles = []
    for x in np.arange(xmin, xmax, grid_size_deg):
        for y in np.arange(ymin, ymax, grid_size_deg):
            tile = box(x, y, x + grid_size_deg, y + grid_size_deg)
            tiles.append(tile)

    # Create GeoDataFrame for grid
    grid_gdf = gpd.GeoDataFrame(geometry=tiles, crs= GEODETIC_CRS)

    # Keep only tiles intersecting with the country boundary
    grid_gdf = grid_gdf[grid_gdf.intersects(country_gdf)].reset_index(drop=True)
    grid_gdf["tile_id"] = grid_gdf.index

    return grid_gdf

## 2.2 GeoDataFrame Earth Engine FeatureCollection
Earth Engine cannot directly process GeoPandas GeoDataFrames. This function converts a GeoDataFrame into an Earth Engine FeatureCollection, so the geometries and attributes can be uploaded and manipulated within the Earth Engine environment.

In [5]:
def gdf_to_fc(gdf: gpd.GeoDataFrame) -> ee.FeatureCollection:
    """
    Convert a grid GeoDataFrame (GEODETIC_CRS = EPSG:4326) to an Earth Engine FeatureCollection
    enriched with centroid lon/lat and SRTM elevation (meters).

    Expects a `tile_id` column.

    Parameters
    ----------
    gdf : geopandas.GeoDataFrame
        Grid in lon/lat (GEODETIC_CRS = EPSG:4326) with Polygon geometries and `tile_id`.

    Returns
    -------
    ee.FeatureCollection
    """
    # This dataset contains a Digital Elevation Model (DEM) with a spatial resolution of 30 meters.
    dem = ee.Image('USGS/SRTMGL1_003') 

    def make_feature(row) -> ee.Feature:
        tile_geom = ee.Geometry.Polygon(row['geometry'].__geo_interface__['coordinates'])
        centroid = tile_geom.centroid()
        return ee.Feature(tile_geom, {
            'tile_id': int(row['tile_id']),
            'centroid_lon': centroid.coordinates().get(0),
            'centroid_lat': centroid.coordinates().get(1)
        })

    # Convert to EE FeatureCollection without elevation first
    features = [make_feature(row) for _, row in gdf.iterrows()]
    fc = ee.FeatureCollection(features)

    # Add elevation in EE using .map() by sampling at the centroid
    def add_elevation(feature: ee.Feature) -> ee.Feature:
        centroid = ee.Geometry.Point([
            feature.get('centroid_lon'),
            feature.get('centroid_lat')
        ])
        elev = dem.sample(centroid, 30).first()

        # Use default -9999 if sampling fails (e.g., over oceans)
        alt = ee.Algorithms.If(elev, elev.get('elevation'), -9999)
        return feature.set('centroid_alt', alt)

    return fc.map(add_elevation)

# Datasets and Date Window
- **Normalized Difference Vegetation Index (NDVI)**  
  Source: *MODIS MOD13Q1 v6.1*, 16-day composite, 250 m spatial resolution.  
  The NDVI band is stored as `int16` values scaled by **1 × 10⁻⁴**, representing vegetation greenness (−1 to +1).

- **Soil Moisture Active Passive (SMAP)**  
  Source: *NASA SMAP SPL3SMP_E* (Enhanced Level-3 Soil Moisture, ~9 km resolution).  
  The product transitioned from **version 005** to **version 006** on **2023-12-04**;  
  both versions are merged to ensure a continuous soil-moisture time series.

- **Fire Information for Resource Management System (FIRMS)**  
  Source: *NASA FIRMS* active fire detections derived from **MODIS** observations.  
  We use **thermal band 21 (T21)** to represent the brightness temperature of detected fire pixels.

In [ ]:
# Date window
start_date, end_date = '2020-01-01', '2025-03-31'

# ======================================================================================
# MODIS NDVI (MOD13Q1 v6.1)
ndvi = (
    ee.ImageCollection("MODIS/061/MOD13Q1")
    .filterDate(start_date, end_date)
    .select('NDVI')
)
# ======================================================================================

# ======================================================================================
# SMAP Soil Moisture (SPL3SMP_E)
# The SMAP Enhanced Level-3 Soil Moisture product transitioned from version 005 to 006.
# - Data up to 2023-12-03 are available in the v005 collection.
smap_v5  = (
    ee.ImageCollection("NASA/SMAP/SPL3SMP_E/005")
    .filterDate(start_date, '2023-04-30')
)
# - Data starting from 2023-12-04 onward are available in the v006 collection.
smap_v6  = (
    ee.ImageCollection("NASA/SMAP/SPL3SMP_E/006")
    .filterDate('2023-05-01', end_date)
)
# To build a continuous time series, we load both versions and merge them.
sm = (
    smap_v5.merge(smap_v6)
    .select(['soil_moisture_am', 'soil_moisture_pm'])
)
# ======================================================================================

# ======================================================================================
# Load FIRMS T21
# The brightness temperature of a fire pixel using MODIS channels 21/22 (T21)
viirs = (
    ee.ImageCollection("FIRMS")
    .filterDate(start_date, end_date)
    .select('T21')
)
# ======================================================================================

# 3. Import, process, and export
This section reduces satellite products over a precomputed grid of the country of interest (Portugal) and exports the results to Google Drive.

**Datasets**
- **NDVI**: MODIS MOD13Q1 v6.1 (16-day composite)
- **FIRMS**: Active fire product (brightness temperature band selection as in code)
- **SMAP**: SPL3SMP_E soil moisture (AM/PM)

## 3.1 Load the geometry of the country as a FeatureCollection
We load a precomputed Portugal grid from a local GeoJSON file (source: [SimpleMaps – Portugal GIS data](https://simplemaps.com/gis/country/pt#all)) and convert it to an `ee.FeatureCollection`, enabling server-side reductions in Google Earth Engine.


In [ ]:
import os
portugal_grid_gdf = gpd.read_file(os.path.join('data', 'geojson', 'portugal_grid.geojson'))
portugal_grid_fc = gdf_to_fc(portugal_grid_gdf)

## 3.2 NDVI Per-Tile, Reduction, and Export
Here, we:
- Map a reducer over each NDVI image,
- Attach the image date and grid attributes,
- Flatten the output into a single `FeatureCollection`,
- Export the table to Google Drive.

In [ ]:
# NDVI per-tile extraction function
def extract_per_tile(image):
    image = image.unmask(None)  # fill masked pixels with 0 before reduceRegions

    reduced = image.reduceRegions(
        collection=portugal_grid_fc,
        reducer=ee.Reducer.mean().setOutputs(['avg_NDVI']),
        scale=1000,  # 061/MODIS native resolution 1000
    )

    date = image.date().format('YYYY-MM-dd')

    def format_feature(f):
        return f.set({
            'date': date,
            'avg_NDVI': f.get('avg_NDVI'),
            'tile_id': f.get('tile_id'),
            'centroid_lon': f.get('centroid_lon'),
            'centroid_lat': f.get('centroid_lat'),
            'centroid_alt': f.get('centroid_alt')
        })

    return reduced.map(format_feature)

# Apply the function to each image in the collection
features = ndvi.map(extract_per_tile).flatten()

### A quick test!
Running the snippet below fetches a few features to confirm that the per-tile extraction is producing the expected properties (`date`, metrics, `tile_id`, centroid fields).  
> This call triggers server-side computation and can take a bit of time. Use a small `limit(...)` to keep the response lightweight and to quickly catch mistakes in reducers, band names, or scales.

> Note: `.getInfo()` collects computations and blocks until results return; the `limit(5)` keeps it fast by limiting to the 5 first features.

In [ ]:
# JUST FOR A QUICK TEST --- Inspect the properties of the first few features
first_few_features = features.limit(5).getInfo()
if first_few_features and 'features' in first_few_features:
    for feature in first_few_features['features']:
        print(f"Feature properties: {feature['properties']}")
else:
    print("No features found in the 'features' collection.")

### Export to Google Drive
If the auick test looks good, run the export below.  
This sends the **server-side** `FeatureCollection` to your Google Drive as a CSV.

**Timing notes**
- Actual duration varies with the Earth Engine task queue, dataset size, export I/O, and Drive latency.
- Because computation runs on Earth Engine servers, results are largely independent of your local machine’s specs; **however**, server load and quotas can still make runs faster or slower across sessions/computers.
- Monitor progress in the EE **Tasks** panel or by calling `ndvi_task.status()`.  
  Typical progression: `UNSUBMITTED -> READY -> RUNNING -> COMPLETED`.  
  Once the state is `COMPLETED`, the file will appear in Google Drive under the folder **`fire_EarthEngine`** (as configured).

In [ ]:
# Export to Google Drive
ndvi_task = ee.batch.Export.table.toDrive(
    collection=features,
    description='ndvi_portugal',
    folder='fire_EarthEngine',
    fileFormat='CSV'
)
ndvi_task.start()

print("Export started. Check your Google Drive > fire_EarthEngine folder.")

In [ ]:
ndvi_task.status()

## 3.2 FIRMS/T21 Per-Tile, Reduction, and Export
The following block computes the maximum brightness temperature (T21) per grid tile and exports the results to Google Drive.

In [ ]:
# ======================================================================================
# Update the per-tile extraction function for FIRMS T21
# ======================================================================================
def extract_per_tile(image):
    image = image.unmask(None)  # fill masked pixels with 0 before reduceRegions

    reduced = image.reduceRegions(
        collection=portugal_grid_fc,
        reducer=ee.Reducer.max().setOutputs(['max_T21']),
        scale=1000,  # MODIS native resolution 1000
    )

    date = image.date().format('YYYY-MM-dd')

    def format_feature(f):
        return f.set({
            'date': date,
            'max_T21': f.get('max_T21'),
            'tile_id': f.get('tile_id'),
            'centroid_lon': f.get('centroid_lon'),
            'centroid_lat': f.get('centroid_lat'),
            'centroid_alt': f.get('centroid_alt')
        })

    return reduced.map(format_feature)

# Apply the function to each image in the collection
features = viirs.map(extract_per_tile).flatten()

### Quick test

In [ ]:
# JUST FOR A QUICK TEST --- Inspect the properties of the first few features
first_few_features = features.limit(5).getInfo()
if first_few_features and 'features' in first_few_features:
    for feature in first_few_features['features']:
        print(f"Feature properties: {feature['properties']}")
else:
    print("No features found in the 'features' collection.")

### Export to Google Drive

In [ ]:
# Export to Google Drive (keep this part)
viirs_task = ee.batch.Export.table.toDrive(
    collection=features,
    description='FIRMS_Fire_Data_Per_Tile_2020-2025_with_alt',
    folder='fire_EarthEngine',
    fileFormat='CSV'
)
viirs_task.start()

print("Export started. Check your Google Drive > fire_EarthEngine folder.")

In [ ]:
viirs_task.status()

## 3.3 Soil Moisture Per-Tile, Reduction, and Export

In [ ]:
# ======================================================================================
# Update the per-tile extraction function for SMAP Soil Moisture
# ======================================================================================
def extract_per_tile(image):
    image = image.unmask(None)  # fill masked pixels with 0 before reduceRegions

    reduced = image.reduceRegions(
        collection=portugal_grid_fc,
        reducer=ee.Reducer.mean().setOutputs(['soil_moisture_am']),
        scale=9000 # Native resolution 9000
    )

    date = image.date().format('YYYY-MM-dd')

    def format_feature(f):
        return f.set({
            'date': date,
            'soil_moisture_am': f.get('soil_moisture_am'),
            'soil_moisture_pm': f.get('soil_moisture_pm'),
            'tile_id': f.get('tile_id'),
            'centroid_lon': f.get('centroid_lon'),
            'centroid_lat': f.get('centroid_lat'),
            'centroid_alt': f.get('centroid_alt')
        })

    return reduced.map(format_feature)


# Apply the function to each image in the collection
features = sm.map(extract_per_tile).flatten()

### Quick test

In [ ]:
# JUST FOR A QUICK TEST --- Inspect the properties of the first few features
first_few_features = features.limit(5).getInfo()
if first_few_features and 'features' in first_few_features:
    for feature in first_few_features['features']:
        print(f"Feature properties: {feature['properties']}")
else:
    print("No features found in the 'features' collection.")

### Export to Google Drive

In [ ]:
# Export to Google Drive (keep this part)
sm_task = ee.batch.Export.table.toDrive(
    collection=features,
    description='sm_am_pm_Per_Tile_2020-2025_with_alt',
    folder='EarthEngine',
    fileFormat='CSV'
)
sm_task.start()

print("Export started. Check your Google Drive > EarthEngine folder.")

In [ ]:
sm_task.status()